## Implementing 1D Convolution (Cross-Correlation) in PyTorch — With Stride

This notebook demonstrates how to implement **1D convolution** (actually **cross-correlation**) in PyTorch from scratch using two methods:

1. **Loop-based implementation** (educational, intuitive)
2. **Vectorized implementation** (fast, efficient)


### Problem Setup

We are given:

- Input signal `x` of length `N`
- Kernel/filter `w` of length `K`
- Stride `s` (default = 1)


The **cross-correlation** (used in PyTorch) is defined as:

$$
y[i] = \sum_{j=0}^{K-1} x[i + j] \cdot w[j]
$$


The output length is computed as:

$$
\text{output\_length} = \left\lfloor \frac{N - K}{s} \right\rfloor + 1
$$

### Example Inputs

```python
import torch

x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])  # Input signal (length 5)
w = torch.tensor([0.2, 0.5, 0.3])           # Kernel (length 3)
stride = 2

## Method 1: Loop-Based Simple Implementation

In [2]:
import torch

def conv1d_loop_stride(x, w, stride=1):
    """
    Performs 1D cross-correlation using explicit loops and stride.

    Args:
        x (torch.Tensor): Input signal (1D tensor)
        w (torch.Tensor): Kernel/filter (1D tensor)
        stride (int): Stride value

    Returns:
        torch.Tensor: Output after cross-correlation
    """
    K = len(w)
    N = len(x)
    output_len = ((N - K) // stride) + 1  # Compute output length
    y = torch.zeros(output_len)

    for i in range(output_len):
        start = i * stride  # Start index for current window
        for j in range(K):
            y[i] += x[start + j] * w[j]  # Elementwise product and sum
    return y

# Example usage
x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
w = torch.tensor([0.2, 0.5, 0.3])
print(conv1d_loop_stride(x, w, stride=2))  # tensor([2.1, 4.1])

tensor([2.1000, 4.1000])


## Method 2: Vectorized Implementation

In [3]:
def conv1d_vectorized_stride(x, w, stride=1):
    """
    Performs 1D cross-correlation using vectorized PyTorch operations with stride.

    Args:
        x (torch.Tensor): Input signal (1D tensor)
        w (torch.Tensor): Kernel/filter (1D tensor)
        stride (int): Stride value

    Returns:
        torch.Tensor: Output after cross-correlation
    """
    K = len(w)
    # Use unfold to extract sliding windows with given stride
    x_unfolded = x.unfold(0, K, stride)  # Shape: (output_len, K)
    y = (x_unfolded * w).sum(dim=1)      # Elementwise multiply and sum over K
    return y

# Example usage
print(conv1d_vectorized_stride(x, w, stride=2))  # tensor([2.1, 4.1])

tensor([2.1000, 4.1000])


## Method 3: PyTorch Built-in `Conv1d`

In [14]:
import torch.nn as nn

# PyTorch expects input shape: (batch_size, in_channels, length)
x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
x_torch = x.reshape(1, 1, x.size(0)) # Create a fake batch dimension Shape: (1, 1, 5)

# Create Conv1d layer manually with weights and stride
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=2, bias=False)

# Forward pass
y = conv(x_torch)             # Shape: (1, 1, output_len)
print(y.squeeze())

tensor([0.3461, 0.0479], grad_fn=<SqueezeBackward0>)


## 2D Convolution: PyTorch Built-in `Conv2d`

In [11]:
import torch.nn as nn

# PyTorch expects input shape: (batch_size, in_channels, height, width)
x_torch = torch.rand(5, 3, 16, 16) # shape: (batch_size=5, in_channels=3, height=16, width=16)

# Create Conv2d layer
conv = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1)

# Forward pass
y = conv(x_torch)
print(y.shape) # Shape: (5, 8, 14, 14)

torch.Size([5, 8, 14, 14])
